In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.models import TCNModel
from darts.dataprocessing.transformers import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from darts.utils.likelihood_models import QuantileRegression
import sys
import json
sys.path.append('../Helper/')
from dataPreprocessing import get_untransformed_exog, TRAIN_DATA_PATH_1990S, TEST_DATA_PATH_1990S
from PyTorchModular import darts_optuna,save_model_hyper_params, load_model_hyper_params, train_valid_split_darts, load_prediction, HORIZONS
from torch.nn import MSELoss
from torch.optim import AdamW
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import numpy as np
import os

In [ ]:
target_col = 'fred_PCEPI'
date_col = 'observation_date'

In [ ]:
def get_exog_target(path : str, date_col : str, target_col : str):
    data = pd.read_csv(path)
    data = get_untransformed_exog(data)
    df = data.copy()
    df[date_col] = pd.to_datetime(df[date_col], format='%m/%Y')
    df.set_index(date_col, inplace=True)
    return df.drop(target_col, axis=1), df[target_col]

In [ ]:
train_exog, train_target = get_exog_target(TRAIN_DATA_PATH_1990S, date_col, target_col)

test_exog, test_target = get_exog_target(TEST_DATA_PATH_1990S, date_col, target_col)

In [ ]:
N_TRIALS=100

In [ ]:
search_space = {
    'kernel_size': (int, 1, 6),
    'num_filters': (int, 1, 10),
    'num_layers': (int, 1, 10),
    'dilation_base': (int, 1, 5),
    'weight_norm': (str, [False, True]),
    'dropout': (float, 0.0, 1.0),
    'optimizer_kwargs': {"lr": (float, 1e-5, 1e-1,{'step':None,'log':True})},
    'loss_fn':('categorical',["QuantileRegression", "MSE"])
}

In [ ]:
def load_TCN_params(json_path):

    with open(json_path,'r') as f:
        loaded_params= json.load(f)

    if 'loss_fn' in loaded_params.keys():
        if loaded_params['loss_fn']=='MSE':
            loaded_params['loss_fn']=MSELoss()
            loaded_params['likelihood']=None
        else:
            loaded_params['loss_fn']=None
            loaded_params['likelihood']=QuantileRegression((0.25,0.5,0.75))

    loaded_params['optimizer_kwargs']={"lr":loaded_params["lr"]}
    del loaded_params["lr"]

    return loaded_params


def train_TCN(best_params_path, valid_size, horizon, train_exog, train_target, verbose=False):
    
    best_params= load_TCN_params(best_params_path)
    target_scaler = Scaler()
    exog_scaler = Scaler()

    probabilistic= True if best_params['loss_fn'] is None else False

    train_target, valid_target, train_exog, valid_exog = train_valid_split_darts(train_exog, train_target, valid_size, best_params['input_chunk_length'])

    train_target= target_scaler.fit_transform(train_target)
    train_exog= exog_scaler.fit_transform(train_exog)

    valid_target= target_scaler.transform(valid_target)
    valid_exog= exog_scaler.transform(valid_exog)

    early_stopper = EarlyStopping(
                monitor="val_loss",
                patience=25,
                min_delta=1e-5,
                mode="min",
                verbose=verbose
                )
    best_params["save_checkpoints"]=True
    best_params["force_reset"]=True
    best_params["random_state"]=42
    best_params["output_chunk_length"]=horizon
    best_params['optimizer_cls']=AdamW
    best_params["pl_trainer_kwargs"]={
                "accelerator":'auto',
                "callbacks": [early_stopper]
            }

    model = TCNModel(**best_params, model_name= f'TCN_horizon_{horizon}')

    model.fit(train_target,
            past_covariates=train_exog,
            val_series=valid_target,
            val_past_covariates=valid_exog,
            epochs=10000,
            verbose=verbose)

    model.load_from_checkpoint(model_name=f'TCN_horizon_{horizon}', best=True)

    return model, target_scaler, exog_scaler, best_params['input_chunk_length'], probabilistic

def infer_darts(model,train_exog_df, train_target_series, test_exog_df, test_target_series,exog_scaler,target_scaler, input_size, horizon, probabilistic=False):
    
    combined_exog_df= pd.concat((train_exog_df,test_exog_df),axis=0)
    combined_target_series= pd.concat((train_target_series,test_target_series),axis=0)

    preds_list=np.array([])

    for i in range(0,12,horizon):
        x,y=load_prediction(input_size,combined_exog_df,combined_target_series,exog_scaler,target_scaler,test_target.index[i])
        if probabilistic:
            pred=target_scaler.inverse_transform(model.predict(n=horizon,series=y,past_covariates=x,verbose=False, num_samples=1000)).values().flatten()
        else:
            pred=target_scaler.inverse_transform(model.predict(n=horizon,series=y,past_covariates=x,verbose=False)).values().flatten()

        print(test_target.index[i:i+horizon])
        print(pred)
        preds_list= np.append(preds_list,pred)
    
    return preds_list

In [ ]:
def TNC_tune_and_save_horizons(horizons, n_trials, verbose=False, use_existing_params=True):
    best_params_path = os.path.join('.', 'bestHyperparameters')
    valid_sizes = []
    for horizon in horizons:
        if verbose: print(f'Optimising for horizon {horizon}...')
        existing_params = []
        valid_size = horizon+3
        invariates = {
            'output_chunk_length':horizon,
            'optimizer_cls':AdamW
        }
        valid_sizes.append(valid_size)
        cur_hyperparams_f = os.path.join(best_params_path, f'best_params_tcn_horizon_{horizon}.json')
        # Assumes TNC search space and adapts depending on horizon
        search_space['input_chunk_length'] = (int, 7 if horizon < 7 else horizon+1, 36,{'step':1,'log':False})
        if use_existing_params: existing_params = [load_model_hyper_params(cur_hyperparams_f)]
        best_params=darts_optuna(TCNModel, 'TCN', search_space, invariates, train_target, train_exog, valid_size, horizon, n_trials=n_trials, patience=5, tol=1e-4,
                                 existing_params=existing_params, verbose=verbose)
        save_model_hyper_params(cur_hyperparams_f, best_params)
    return valid_sizes

## Horizon = 1

In [ ]:
valid_sizes = []

In [ ]:
VERBOSE = False

In [ ]:
best_params_path = os.path.join('.', 'bestHyperparameters')

In [ ]:
HORIZON=1
VALID_SIZE=HORIZON+3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW
}
valid_sizes.append(VALID_SIZE)
cur_hyperparams_f = os.path.join(best_params_path, f'best_params_tcn_horizon_{HORIZON}.json')

In [ ]:
best_params=darts_optuna(TCNModel, 'TCN', search_space, invariates, train_target, train_exog, VALID_SIZE, HORIZON, n_trials=N_TRIALS, patience=5, tol=1e-4,
                         existing_params=[load_model_hyper_params(cur_hyperparams_f)], verbose=True)
save_model_hyper_params(cur_hyperparams_f, best_params)

### Horizon = 3

In [ ]:
HORIZON=3
VALID_SIZE=HORIZON+3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW
}
valid_sizes.append(VALID_SIZE)
cur_hyperparams_f = os.path.join(best_params_path, f'best_params_tcn_horizon_{HORIZON}.json')

In [ ]:
best_params=darts_optuna(TCNModel, 'TCN', search_space, invariates, train_target, train_exog, VALID_SIZE, HORIZON, n_trials=N_TRIALS, patience=5, tol=1e-4,
                         existing_params=[load_model_hyper_params(cur_hyperparams_f)], verbose=True)
save_model_hyper_params(cur_hyperparams_f, best_params)

### Biannual Horizon

In [ ]:
HORIZON=6
VALID_SIZE=HORIZON+3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW
}
valid_sizes.append(VALID_SIZE)
cur_hyperparams_f = os.path.join(best_params_path, f'best_params_tcn_horizon_{HORIZON}.json')

In [ ]:
best_params=darts_optuna(TCNModel, 'TCN', search_space, invariates, train_target, train_exog, VALID_SIZE, HORIZON, n_trials=N_TRIALS, patience=5, tol=1e-4,
                         existing_params=[load_model_hyper_params(cur_hyperparams_f)], verbose=True)
save_model_hyper_params(cur_hyperparams_f, best_params)

### Annual Horizon

In [ ]:
HORIZON=6
VALID_SIZE=HORIZON+3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW
}
search_space['input_chunk_length'] = (int, 7 if HORIZON < 7 else HORIZON+1, 36,{'step':1,'log':False})
valid_sizes.append(VALID_SIZE)
cur_hyperparams_f = os.path.join(best_params_path, f'best_params_tcn_horizon_{HORIZON}.json')

In [ ]:
best_params=darts_optuna(TCNModel, 'TCN', search_space, invariates, train_target, train_exog, VALID_SIZE, HORIZON, n_trials=N_TRIALS, patience=5, tol=1e-4,
                         existing_params=[load_model_hyper_params(cur_hyperparams_f)], verbose=True)
save_model_hyper_params(cur_hyperparams_f, best_params)

### All Horizons (skip to here to train all at once)

In [ ]:
# Can find best hyperparameters for all horizons using this line
valid_sizes = valid_sizes + TNC_tune_and_save_horizons(HORIZONS, N_TRIALS, verbose=True, use_existing_params=True)

### Train Horizons with Best Hyperparameters and Make Predictions

In [ ]:
# For initialising validation sizes if haven't optimised in the same runtime as making predictions
#valid_sizes = [horizon+3 for horizon in HORIZONS]

In [ ]:
preds_root = os.path.join('..', '..', 'Predictions')

In [ ]:
for i, horizon in enumerate(HORIZONS):
    cur_model, target_scaler, exog_scaler, input_size, probabilistic = train_TCN(os.path.join(best_params_path, f'best_params_tcn_horizon_{horizon}.json'),
                                                                                 valid_size=valid_sizes[i], horizon=horizon, train_exog=train_exog, train_target=train_target,
                                                                                 verbose=True)
    preds = infer_darts(cur_model, train_exog, train_target,
                        test_exog, test_target, exog_scaler=exog_scaler,
                        target_scaler=target_scaler, input_size=input_size,
                        horizon=horizon, probabilistic=probabilistic)

    np.save(os.path.join(preds_root, f'Horizon{horizon}', f'TCN_horizon_{horizon}.npy'), preds)